In [4]:
import sys

sys.path.append("./reference")

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Reload models

In [98]:
from PIL import Image
from IPython.display import display
import torch as th

from reference.glide_text2im.download import load_checkpoint
from reference.glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)

device = 'cpu'

# Create ref model.
options = model_and_diffusion_defaults()
options['use_fp16'] = False
options['timestep_respacing'] = '2' # use 100 diffusion steps for fast sampling
ref_model, diffusion = create_model_and_diffusion(**options)
ref_model.eval()
ref_model.to(device)
ref_model.load_state_dict(load_checkpoint('base', device))
print('total base parameters', sum(x.numel() for x in ref_model.parameters()))

# Create our model
from unet import Text2Im
from reference.glide_text2im.tokenizer.bpe import get_encoder
tokenizer = get_encoder()
our_model = Text2Im(tokenizer)
state = ref_model.state_dict()
mapped = {ours: state[theirs] for ours, theirs in zip(our_model.state_dict().keys(), ref_model.state_dict().keys())}
our_model.load_state_dict(mapped)

total base parameters 385030726


<All keys matched successfully>

In [94]:
def show_images(batch: th.Tensor):
    """ Display a batch of images inline. """
    scaled = ((batch + 1)*127.5).round().clamp(0,255).to(th.uint8).cpu()
    reshaped = scaled.permute(2, 0, 3, 1).reshape([batch.shape[2], -1, 3])
    display(Image.fromarray(reshaped.numpy()))

In [86]:
##############################
# Sample from the base model #
##############################

prompt = 'cat'
batch_size = 1

# Create the text tokens to feed to the model.
tokens = ref_model.tokenizer.encode(prompt)
tokens, mask = ref_model.tokenizer.padded_tokens_and_mask(
    tokens, options['text_ctx']
)



# Pack the tokens together into model kwargs.
model_kwargs = dict(
    tokens=th.tensor(
        [tokens] * batch_size, device=device
    ),
    mask=th.tensor(
        [mask] * batch_size,
        dtype=th.bool,
        device=device,
    ),
)

In [87]:
img = th.randn([1, 3, 64, 64])


In [89]:
ref_out = ref_model(copy.deepcopy(img), timesteps, **model_kwargs)[:,:3]


TimestepEmbedSequential(
  (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
TimestepEmbedSequential(
  (0): ResBlock(
    (in_layers): Sequential(
      (0): GroupNorm32(32, 192, eps=1e-05, affine=True)
      (1): Identity()
      (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (h_upd): Identity()
    (x_upd): Identity()
    (emb_layers): Sequential(
      (0): SiLU()
      (1): Linear(in_features=768, out_features=384, bias=True)
    )
    (out_layers): Sequential(
      (0): GroupNorm32(32, 192, eps=1e-05, affine=True)
      (1): SiLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (skip_connection): Identity()
  )
)
TimestepEmbedSequential(
  (0): ResBlock(
    (in_layers): Sequential(
      (0): GroupNorm32(32, 192, eps=1e-05, affine=True)
      (1): Identity()
      (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [90]:
our_out = our_model(copy.deepcopy(img), timesteps, **model_kwargs)[:,:3]


Wrapper(
  (module): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ResidualBlock(
  (in_layers): Sequential(
    (0): Sequential(
      (0): GroupNorm(32, 192, eps=1e-05, affine=True)
      (1): SiLU()
    )
    (1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (emb_layers): Sequential(
    (0): SiLU()
    (1): Linear(in_features=768, out_features=384, bias=True)
  )
  (group_norm): GroupNorm(32, 192, eps=1e-05, affine=True)
  (out_layers): Sequential(
    (0): SiLU()
    (1): Dropout(p=0.0, inplace=False)
    (2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)
ResidualBlock(
  (in_layers): Sequential(
    (0): Sequential(
      (0): GroupNorm(32, 192, eps=1e-05, affine=True)
      (1): SiLU()
    )
    (1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (emb_layers): Sequential(
    (0): SiLU()
    (1): Linear(in_features=768, out_features=384, bias=True)
  )
  (group_norm): GroupN

In [97]:
from PIL import Image
from IPython.display import display
import torch as th

from reference.glide_text2im.download import load_checkpoint
from reference.glide_text2im.model_creation import (
    create_model_and_diffusion,
    model_and_diffusion_defaults,
)

device = 'cpu'

# Create ref model.
options = model_and_diffusion_defaults()
options['use_fp16'] = False
options['timestep_respacing'] = '2' # use 100 diffusion steps for fast sampling
ref_model, diffusion = create_model_and_diffusion(**options)
ref_model.eval()
ref_model.to(device)
ref_model.load_state_dict(load_checkpoint('base', device))
print('total base parameters', sum(x.numel() for x in ref_model.parameters()))

# Create our model
from unet import Text2Im
from reference.glide_text2im.tokenizer.bpe import get_encoder
tokenizer = get_encoder()
our_model = Text2Im(tokenizer)
state = ref_model.state_dict()
mapped = {ours: state[theirs] for ours, theirs in zip(our_model.state_dict().keys(), ref_model.state_dict().keys())}
our_model.load_state_dict(mapped)


# ----------

import copy

timesteps = th.ones([1])
ref_out = ref_model(copy.deepcopy(img), timesteps, **model_kwargs)[:,:3]
our_out = our_model(copy.deepcopy(img), timesteps, **model_kwargs)[:,:3]

# print('Original noise')
# show_images(copy.deepcopy(img))
# print('Reference')
# show_images(ref_img)
# print('Ours')
# show_images(our_img)

th.allclose(ref_out, our_out)

print(ref_out.shape)

torch.Size([1, 3, 8, 8])


In [49]:
our_model.in_layers[0]


Wrapper(
  (module): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [50]:
ref_model.input_blocks[0]

TimestepEmbedSequential(
  (0): Conv2d(3, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [53]:
our_model.in_layers[0]

TypeError: 'Wrapper' object is not subscriptable